In [18]:
import os


# main db connection data 
NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


# ontology db connection data
NEO4J_URI_ONTOLOGY = os.environ['NEO4J_URI_ONTOLOGY']
NEO4J_USERNAME_ONTOLOGY = os.environ['NEO4J_USERNAME_ONTOLOGY']
NEO4J_PASSWORD_ONTOLOGY = os.environ['NEO4J_PASSWORD_ONTOLOGY']
NEO4J_ONTOLOGY_DB_NAME = os.environ['NEO4J_ONTOLOGY_DB_NAME']
NEO4J_DB_NAME = os.environ['NEO4J_DB_NAME']

In [19]:
from neo4j import GraphDatabase
driver_main = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
driver_ontology = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [20]:
def genPatternDefinedLabelInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (mdl:PatternDefinedLabel)
        RETURN mdl.name AS name, mdl.pattern AS pattern, mdl.classElementVariable AS classElementVariable""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH {record['pattern']} SET {record['classElementVariable']}:{record['name']}"
        yield query

for q in genPatternDefinedLabelInferenceQueries():
    print(q)


MATCH (p:Person) WHERE EXISTS {(p)-[:DIRECTED]->()} SET p:_PersonDirectedSome
MATCH (p:Person) WHERE EXISTS {(p)-[:ACTED_IN]->()} SET p:_PersonActedInSome


In [21]:
def genPatternDefinedRelationshipInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (mdr:PatternDefinedRelationship)
        RETURN mdr.name AS name, mdr.pattern AS pattern, mdr.sourceElementVariable AS sourceElementVariable, mdr.targetElementVariable AS targetElementVariable""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH {record['pattern']} MERGE ({record['sourceElementVariable']})-[:{record['name']}]->({record['targetElementVariable']})"
        yield query

for q in genPatternDefinedRelationshipInferenceQueries():
    print(q)


MATCH (s:Person)-[:INVOLVED_IN]->()<-[:INVOLVED_IN]-(t:Person) MERGE (s)-[:_COLLABORATOR]->(t)
MATCH (s:Person)-[:ACTED_IN]->()<-[:ACTED_IN]-(t:Person) MERGE (s)-[:_COACTOR]->(t)


In [22]:
def genSCOLabelInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (narrower:Label)-[:SCO]->(broader:Label)
        RETURN narrower.name AS narrower, broader.name AS broader""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n:{record['narrower']}) SET n:{record['broader']}"
        yield query

for q in genSCOLabelInferenceQueries():
    print(q)

MATCH (n:_KevinBacon) SET n:Actor
MATCH (n:Actor) SET n:Person
MATCH (n:Director) SET n:Person


In [23]:
def genImpliesRelationshipInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (narrower:Relationship)-[:IMPLIES]->(broader:Relationship)
        RETURN narrower.name AS narrower, broader.name AS broader""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n)-[:{record['narrower']}]->(m) MERGE (n)-[:{record['broader']}]->(m)"
        yield query

for q in genImpliesRelationshipInferenceQueries():
    print(q)

MATCH (n)-[:ACTED_IN]->(m) MERGE (n)-[:INVOLVED_IN]->(m)
MATCH (n)-[:DIRECTED]->(m) MERGE (n)-[:INVOLVED_IN]->(m)


In [24]:
def genEquivalentLabelInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (l1:Label)-[:EQUIVALENT]->(l2:Label)
        RETURN l1.name AS l1, l2.name AS l2""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n:{record['l1']}|{record['l2']}) SET n:{record['l1']}:{record['l2']}"
        yield query

for q in genEquivalentLabelInferenceQueries():
    print(q)

MATCH (n:Actor|_PersonActedInSome) SET n:Actor:_PersonActedInSome
MATCH (n:Director|_PersonDirectedSome) SET n:Director:_PersonDirectedSome


In [25]:
def genEquivalentRelationshipInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (r1:Relationship)-[:EQUIVALENT]->(r2:Relationship)
        RETURN r1.name AS r1, r2.name AS r2""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n)-[:{record['r1']}|{record['r2']}]->(m) MERGE (n)-[:{record['r1']}]->(m) MERGE (m)-[:{record['r1']}]->(n)"
        yield query

for q in genEquivalentRelationshipInferenceQueries():
    print(q)

MATCH (n)-[:COLLABORATOR|_COLLABORATOR]->(m) MERGE (n)-[:COLLABORATOR]->(m) MERGE (m)-[:COLLABORATOR]->(n)
MATCH (n)-[:COACTOR|_COACTOR]->(m) MERGE (n)-[:COACTOR]->(m) MERGE (m)-[:COACTOR]->(n)


In [26]:
def genSymmetricRelationshipInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (r:Relationship&Symmetric)
        RETURN r.name AS sim_rel""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n)-[:{record['sim_rel']}]->(m) MERGE (m)-[:{record['sim_rel']}]->(n)"
        yield query

for q in genSymmetricRelationshipInferenceQueries():
    print(q)

MATCH (n)-[:_COLLABORATOR]->(m) MERGE (m)-[:_COLLABORATOR]->(n)
MATCH (n)-[:COLLABORATOR]->(m) MERGE (m)-[:COLLABORATOR]->(n)
MATCH (n)-[:COACTOR]->(m) MERGE (m)-[:COACTOR]->(n)
MATCH (n)-[:_COACTOR]->(m) MERGE (m)-[:_COACTOR]->(n)


In [43]:
def genPatternDefinedNodePropertyInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (n:Label)-[:HAS_PROPERTY]->(qdp:PatternDefinedNodeProperty)
        RETURN n.name AS label, qdp.name AS property_name, qdp.pattern AS pattern, qdp.propertyOwnerVariable AS variable, qdp.valueVariable AS val_variable""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"""MATCH ({record['variable']}:{record['label']})
        CALL ({record['variable']}) {{MATCH {record['pattern']}
        WITH {record['variable']}, {record['val_variable']}
        WHERE {record['variable']}.{record['property_name']} IS NULL OR {record['variable']}.{record['property_name']} <> {record['val_variable']}
        SET {record['variable']}.{record['property_name']} = {record['val_variable']} }}
        IN CONCURRENT TRANSACTIONS OF 10 ROWS"""
        yield query

for q in genPatternDefinedNodePropertyInferenceQueries():
    print(q)

CYPHER runtime=parallel
        MATCH (x:Actor)
        CALL (x) {MATCH SHORTEST 1 (x)-[ca:COACTOR]-*(y:_KevinBacon) WITH ca LIMIT 1 WITH size(ca) AS kbn
        WITH x, kbn
        WHERE x.kb_number IS NULL OR x.kb_number <> kbn
        SET x.kb_number = kbn }
        IN CONCURRENT TRANSACTIONS OF 10 ROWS


In [44]:
inferenceRulesGenerators = [genPatternDefinedLabelInferenceQueries,
                            genPatternDefinedRelationshipInferenceQueries,
                            genSCOLabelInferenceQueries,
                            genImpliesRelationshipInferenceQueries,
                            genEquivalentLabelInferenceQueries,
                            genEquivalentRelationshipInferenceQueries,
                            genSymmetricRelationshipInferenceQueries,
                            genPatternDefinedNodePropertyInferenceQueries]

def genOntologyInferenceQueries(inferenceRulesGenerators):
    for rule in inferenceRulesGenerators:
        for q in rule():
            yield q

list(genOntologyInferenceQueries(inferenceRulesGenerators))

['MATCH (p:Person) WHERE EXISTS {(p)-[:DIRECTED]->()} SET p:_PersonDirectedSome',
 "MATCH (p:Person {name: 'Kevin Bacon'}) SET p:_KevinBacon",
 'MATCH (p:Person) WHERE EXISTS {(p)-[:ACTED_IN]->()} SET p:_PersonActedInSome',
 'MATCH (s:Person)-[:INVOLVED_IN]->()<-[:INVOLVED_IN]-(t:Person) MERGE (s)-[:_COLLABORATOR]->(t)',
 'MATCH (s:Person)-[:ACTED_IN]->()<-[:ACTED_IN]-(t:Person) MERGE (s)-[:_COACTOR]->(t)',
 'MATCH (n:_KevinBacon) SET n:Actor',
 'MATCH (n:Actor) SET n:Person',
 'MATCH (n:Director) SET n:Person',
 'MATCH (n)-[:ACTED_IN]->(m) MERGE (n)-[:INVOLVED_IN]->(m)',
 'MATCH (n)-[:DIRECTED]->(m) MERGE (n)-[:INVOLVED_IN]->(m)',
 'MATCH (n:Actor|_PersonActedInSome) SET n:Actor:_PersonActedInSome',
 'MATCH (n:Director|_PersonDirectedSome) SET n:Director:_PersonDirectedSome',
 'MATCH (n)-[:COLLABORATOR|_COLLABORATOR]->(m) MERGE (n)-[:COLLABORATOR]->(m) MERGE (m)-[:COLLABORATOR]->(n)',
 'MATCH (n)-[:COACTOR|_COACTOR]->(m) MERGE (n)-[:COACTOR]->(m) MERGE (m)-[:COACTOR]->(n)',
 'MATCH (n

In [45]:
def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver_main.session(database=NEO4J_DB_NAME) as session:
                result = session.run(rule, params)
            any_new_update = result.consume().counters._contains_updates
            any_update = any_update or any_new_update
        if not any_update:
            break

In [46]:
infer(list(genOntologyInferenceQueries(inferenceRulesGenerators)))

ClientError: {code: Neo.ClientError.Statement.RuntimeUnsupportedError} {message: The parallel runtime does not support updating queries. Please use another runtime}